In [2]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Define embedding dimensions
age_embedding_dim = 5
gender_embedding_dim = 5
country_embedding_dim = 5

# Define input layers
Year = Input(shape=(1,), dtype='float32', name='Year')
Age = Input(shape=(1,), dtype='int32', name='Age')
Country = Input(shape=(1,), dtype='int32', name='Country')
Gender = Input(shape=(1,), dtype='int32', name='Gender')

# Define embedding layers
Age_embed = Flatten()(Embedding(input_dim=100, output_dim=age_embedding_dim, input_length=1, name='Age_embed')(Age))
Gender_embed = Flatten()(Embedding(input_dim=2, output_dim=gender_embedding_dim, input_length=1, name='Gender_embed')(Gender))
Country_embed = Flatten()(Embedding(input_dim=41, output_dim=country_embedding_dim, input_length=1, name='Country_embed')(Country))

# Concatenate features
features = Concatenate()([Year, Age_embed, Gender_embed, Country_embed])

# Define middle layers
middle = features
dropout_rate = 0.05
for _ in range(4):
    middle = Dense(units=128, activation='tanh')(middle)
    middle = BatchNormalization()(middle)
    middle = Dropout(dropout_rate)(middle)

# Define main output
main_output = Concatenate()([features, middle])
main_output = Dense(units=128, activation='tanh')(main_output)
main_output = BatchNormalization()(main_output)
main_output = Dropout(dropout_rate)(main_output)
main_output = Dense(units=1, activation='sigmoid', name='main_output')(main_output)

# Create model
model = Model(inputs=[Year, Age, Country, Gender], outputs=[main_output])
# Generate some test data
num_samples = 1000
Year_data = np.random.randint(2000, 2021, size=(num_samples, 1)).astype('int32')
Age_data = np.random.randint(0, 100, size=(num_samples, 1)).astype('int32')
Country_data = np.random.randint(0, 41, size=(num_samples, 1)).astype('int32')
Gender_data = np.random.randint(0, 2, size=(num_samples, 1)).astype('int32')
target_data = np.random.randint(0, 2, size=(num_samples, 1)).astype('float32')

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0005), 
              loss='mse')#, metrics=['accuracy'])

lr_callback = ReduceLROnPlateau(factor=0.80, patience=5, verbose=1, cooldown=5, min_lr=0.00005)

mc_callback = ModelCheckpoint(
    filepath='Lee_Carter_NN_model.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False
)


# Fit the model
model.fit(x=[Year_data, Age_data, Country_data, Gender_data], 
          y=target_data, 
          epochs=10, 
          batch_size=32, 
          verbose=0,
          shuffle=True,
          validation_split=0.05,
          callbacks=[mc_callback, lr_callback])


Epoch 1: val_loss improved from inf to 0.25644, saving model to Lee_Carter_NN_model.keras

Epoch 2: val_loss improved from 0.25644 to 0.25497, saving model to Lee_Carter_NN_model.keras

Epoch 3: val_loss did not improve from 0.25497

Epoch 4: val_loss did not improve from 0.25497

Epoch 5: val_loss did not improve from 0.25497

Epoch 6: val_loss did not improve from 0.25497

Epoch 7: val_loss did not improve from 0.25497

Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.

Epoch 8: val_loss improved from 0.25497 to 0.25288, saving model to Lee_Carter_NN_model.keras

Epoch 9: val_loss improved from 0.25288 to 0.25175, saving model to Lee_Carter_NN_model.keras

Epoch 10: val_loss improved from 0.25175 to 0.25087, saving model to Lee_Carter_NN_model.keras
